<a href="https://www.kaggle.com/code/dhirajpant/plantnetv1?scriptVersionId=220895654" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, optimizers, regularizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Add, Multiply, Reshape, Input,ReLU

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [3]:
image_path="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/"

train_path="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/"
valid_path="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/"

test_path="../input/new-plant-diseases-dataset/test/"

In [4]:
# Training data generator with data augmentation
traindata_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Validation data generator (rescaling only)
validdata_generator = ImageDataGenerator(rescale=1./255)

# Test data generator (rescaling only)
testdata_generator=ImageDataGenerator(rescale=1./255)

In [5]:
# Training data
train_generator = traindata_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Validation data
validation_generator = validdata_generator.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Test data
test_generator = testdata_generator.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 33 images belonging to 1 classes.


In [6]:
class_names = os.listdir(image_path)

number_of_images = {}
for class_name in class_names:
    number_of_images[class_name] = len(os.listdir(image_path+"/"+class_name))

images_per_class = pd.DataFrame(list(number_of_images.items()), columns=['Class Name', 'Number of Images'])
images_per_class

,Class Name,Number of Images
0,Tomato___Late_blight,1851
1,Tomato___healthy,1926
2,Grape___healthy,1692
3,Orange___Haunglongbing_(Citrus_greening),2010
4,Soybean___healthy,2022
5,Squash___Powdery_mildew,1736
6,Potato___healthy,1824
7,Corn_(maize)___Northern_Leaf_Blight,1908
8,Tomato___Early_blight,1920
9,Tomato___Septoria_leaf_spot,1745


## Model Design

In [7]:
def se_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // ratio, activation='swish')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    return Multiply()([input_tensor, se])

def plantnet_v1(input_shape=(224, 224, 3), num_classes=38):
    inputs = Input(shape=input_shape)
    
    # Initial Conv Block
    x = Conv2D(32, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = MaxPooling2D((2, 2))(x)

    # Residual Blocks with Fix for Shape Mismatch
    def residual_block(x, filters):
        shortcut = x
        if x.shape[-1] != filters:  # Apply 1x1 Conv if channel mismatch
            shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)

        x = Conv2D(filters, (3, 3), padding='same', activation='swish')(x)
        x = BatchNormalization()(x)
        x = Conv2D(filters, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = se_block(x)  # Squeeze and Excitation
        x = Add()([shortcut, x])  # Residual Connection
        return Activation('swish')(x)

    x = residual_block(x, 64)
    x = MaxPooling2D((2, 2))(x)
    
    x = residual_block(x, 128)
    x = MaxPooling2D((2, 2))(x)
    
    x = residual_block(x, 256)
    x = MaxPooling2D((2, 2))(x)

    # Squeeze and Excitation
    x = se_block(x)

    # Global Average Pooling and Fully Connected Layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='swish')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs, outputs)

# Create and compile the model
model = plantnet_v1()
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 224, 224, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 224, 224, 32)   │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 224, 224, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 112, 112, 32)   │              0 │ activation[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 112, 112, 64)   │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 112, 112, 64)   │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 64)   │         36,928 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 112, 112, 64)   │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 64)             │              0 │ batch_normalization_2… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 4)              │            260 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │            320 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 1, 64)       │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 112, 112, 64)   │          2,112 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 112, 112, 64)   │              0 │ batch_normalization_2… │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 112, 112, 64)   │              0 │ conv2d_1[0][0],        │
│                           │                        │                │ multiply[0][0]         │
├──────────────────────

 Total params: 1,380,946 (5.27 MB)

 Trainable params: 1,379,090 (5.26 MB)

 Non-trainable params: 1,856 (7.25 KB)

In [8]:
# Callbacks
checkpoint = ModelCheckpoint('plantnet_v1_best.keras', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

reduce_lr = LearningRateScheduler(lambda epoch: 0.001 * (0.1 ** (epoch // 10)))


In [9]:
# Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,
    restore_best_weights=True, 
    verbose=1
)

In [ ]:
# Training the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.5812 - loss: 1.4196
Epoch 1: val_accuracy improved from -inf to 0.62844, saving model to plantnet_v1_best.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1076s 476ms/step - accuracy: 0.5813 - loss: 1.4193 - val_accuracy: 0.6284 - val_loss: 1.5682 - learning_rate: 0.0010
Epoch 2/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.9139 - loss: 0.2671
Epoch 2: val_accuracy improved from 0.62844 to 0.70914, saving model to plantnet_v1_best.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 844s 383ms/step - accuracy: 0.9139 - loss: 0.2671 - val_accuracy: 0.7091 - val_loss: 1.1631 - learning_rate: 0.0010
Epoch 3/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.9498 - loss: 0.1550
Epoch 3: val_accuracy improved from 0.70914 to 0.73782, saving model to plantnet_v1_best.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 817s 370ms/step - accuracy: 0.9498 - loss: 0.1550 - val_accuracy: 0.7378 - val_loss: 1.1994 - learning_rate: 0.0010
Epoch 4/50
2197/2

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

# Assuming your model is named 'model'
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
import matplotlib.pyplot as plt

# Sample data extracted from your logs
epochs = list(range(1, 21))
accuracy = [0.5813, 0.9139, 0.9498, 0.9628, 0.9714, 0.9736, 0.9776, 0.9812, 0.9841, 0.9858, 0.9932, 0.9959, 0.9966, 0.9972, 0.9974, 0.9975, 0.9978, 0.9980, 0.9980, 0.9982]
val_accuracy = [0.6284, 0.7091, 0.7378, 0.6960, 0.8860, 0.8467, 0.7727, 0.9166, 0.9569, 0.9062, 0.9856, 0.9826, 0.9890, 0.9803, 0.9795, 0.9796, 0.9836, 0.9571, 0.9804]
loss = [1.4193, 0.2671, 0.1550, 0.1167, 0.0871, 0.0812, 0.0702, 0.0587, 0.0494, 0.0439, 0.0199, 0.0126, 0.0105, 0.0092, 0.0083, 0.0072, 0.0068, 0.0061, 0.0060, 0.0057]
val_loss = [1.5682, 1.1631, 1.1994, 1.9141, 0.5028, 0.5840, 1.1059, 0.6009, 0.1590, 0.7361, 0.0569, 0.0768, 0.0374, 0.1003, 0.1091, 0.0971, 0.0712, 0.2203, 0.0923]

# Plotting accuracy
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, label='Training Accuracy')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()
